In [1]:
#
# RESUME at DataModeler
# 
# Change dm.build_model to dm.load_full_team_dataset
#   This function SHOULD LOAD EVERYTHING 
#  (and all years, adv stats, etc.)
#
# Add dm.set_features
#   This should include years, months (i.e., not playoffs)
#
# Add dm.set_training_params
#   This should include train_ratio, n_models
#
# Add dm.train_model
#   This is basically just the TrainAndTestModel function
#
# 



#from src.DataProcessor import DataProcessor
from src.DataModeler import DataModeler


In [2]:
#dp = DataProcessor()

#dp.scrape_data_player_box_scores()
#dp.scrape_data_team_box_scores()
#dp.scrape_data_season_schedule()

In [3]:
#dp.merge_box_and_season()

In [4]:
dm = DataModeler(start_year = 2007, end_year = 2019)

In [5]:
dm.build_model()


Building model from processed data.

Loading game data for season 2007-2008
Loading game data for season 2008-2009
Loading game data for season 2009-2010
Loading game data for season 2010-2011
Loading game data for season 2011-2012
Loading game data for season 2012-2013
Loading game data for season 2013-2014
Loading game data for season 2014-2015
Loading game data for season 2015-2016
Loading game data for season 2016-2017
Loading game data for season 2017-2018
Loading game data for season 2018-2019
Loading game data for season 2019-2020


In [7]:
dm.team_full_df.tail()

,team,minutes_played,made_field_goals,attempted_field_goals,made_three_point_field_goals,attempted_three_point_field_goals,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,...,turnovers,personal_fouls,location,outcome,game_score,opponent,opponent_score,field_goal_percentage,three_point_percentage,total_rebounds
26985,GOLDEN STATE WARRIORS,240,30,77,6,25,13,18,8,18,...,20,17,home,loss,79.0,SACRAMENTO KINGS,100.0,0.280374,0.193548,26
26986,CHARLOTTE HORNETS,240,28,91,5,32,24,35,17,39,...,10,21,away,loss,85.0,INDIANA PACERS,107.0,0.235294,0.135135,56
26987,INDIANA PACERS,240,40,88,10,35,17,20,8,43,...,9,25,home,win,107.0,CHARLOTTE HORNETS,85.0,0.312500,0.222222,51
26988,ORLANDO MAGIC,240,46,89,17,34,21,26,8,31,...,9,15,away,win,130.0,NEW ORLEANS PELICANS,119.0,0.340741,0.333333,39
26989,NEW ORLEANS PELICANS,240,47,88,19,43,6,12,11,29,...,17,21,home,loss,119.0,ORLANDO MAGIC,130.0,0.348148,0.306452,40
